In [1]:
import numpy as np
import tonic
import tonic.transforms as transforms
import pandas as pd
from torch.utils.data import Dataset, DataLoader
#from tonic.dataset import Dataset
from typing import Callable, Optional
import torch
import matplotlib.pyplot as plt
from torchvision.ops import masks_to_boxes
from torchvision.utils import draw_bounding_boxes
import matplotlib.patches as patches

In [23]:
dir = "/media/user/EVIMO/raw/imo/eval/scene15_dyn_test_01/left_camera/ground_truth_000000"
save_to_dir = "./data/EVIMO/train"
sensor_size = [640, 480, 2]
batch_size = 4
framerate = 200
device = 'cpu'

In [3]:
xy = np.load(dir + "/dataset_events_xy.npy")
polarity = np.load(dir + "/dataset_events_p.npy")
time = np.load(dir + "/dataset_events_t.npy")

In [4]:
print(xy.shape, xy.dtype)
print(time.shape, time.dtype)
print(polarity.shape, polarity.dtype)

(9768348, 2) uint16
(9768348,) float32
(9768348,) uint8


In [5]:
df = pd.DataFrame(columns=["x", "y", "t", "p"])
df["x"] = xy[:, 0]
df["y"] = xy[:, 1]
df["t"] = time
df["p"] = polarity

In [6]:
# Convert timestamp from seconds into microseconds
df["t"] = df["t"].apply(lambda t: t * 1000000)

In [7]:
df

x    y             t  p
0        120   45  5.001785e+03  1
1        334  242  5.003833e+03  1
2        273  110  5.005881e+03  1
3        551   63  5.009721e+03  1
4        515  256  5.009721e+03  0
...      ...  ...           ... ..
9768343   73  230  3.525000e+06  0
9768344  376  230  3.525000e+06  0
9768345  214  239  3.525000e+06  1
9768346  576  239  3.525000e+06  0
9768347   75  253  3.525000e+06  1

[9768348 rows x 4 columns]

In [8]:
events = np.asarray(df.to_records(index=False))

In [9]:
dataset_masks = dict(np.load(dir + "/dataset_mask.npz", allow_pickle=True))

In [10]:
mask_prefix = "mask_0000000"
# Ordering masks, moving to list
masks = []
for i in range(len(dataset_masks)):
    masks.append(dataset_masks[mask_prefix + "{:03d}".format(i)]/1000) # div by 1000 as all object ids are mult by 1000 in original dataset

# Removing extra masks
masks = masks[1:-1]

In [12]:
# Denoise removes isolated, one-off events
# time_window
frame_transform = transforms.Compose([#transforms.Denoise(filter_time=10000),
                                      transforms.ToFrame(sensor_size=sensor_size,
                                                         time_window=5000)
                                     ])

In [13]:
tonic_dataset = EVIMO(data=events, transform=frame_transform, masks=masks)

Target present in: 209 of 703 masks. 0.2972972972972973


In [14]:
binned_events = frame_transform(events)

In [15]:
binned_events.shape

(703, 2, 480, 640)

In [21]:
targets = np.asarray([[0, 1] if 23 in m else [1, 0] for m in masks])

In [22]:
targets.shape

(703, 2)

In [25]:
for i, binned_event in enumerate(binned_events):
    np.save(save_to_dir + "/" + str(i), binned_event)

In [26]:
np.save(save_to_dir + "/labels", targets)